```
    1단계 : 데이터 수집 -> 데이터 출처, 설명
                kNN의 경우 거리를 계산을 해야하므로 데이터가 수치형 데이터였으나 명목형 데이터가 포함되어 있으면 더미코딩을 통해 수치형으로 변환 해야함
                나이브베이즈의 경우는 데이터를 학습하기 위해서 빈도표를 사용하기 때문에 
                행렬을 구성하는 각 클래스와 특징값의 조합을 생성하려면 각 특징이 범주형이어야 한다
```

In [2]:
# 1. 버섯 데이터를 R 로 로드한다. 
mushroom <- read.csv("mushrooms.csv", header=T, stringsAsFactors=T)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
8123,poisonous,knobbed,scaly,brown,no,fishy,free,close,narrow,buff,...,silky,white,white,partial,white,one,evanescent,white,several,leaves


```
    2단계 : 데이터 탐색과 준비 -> 결측, 이상치, 정규분포(히스토그램) 머신러닝 모델이 학습하기 적합한 데이터인지 설명 필요
        명목형이라 확인 안해도 됨
```

```
    3단계 : 데이터 모델로 훈련 -> 나이브베이즈
```

In [ ]:
# 2. 8124 독버섯 데이터만 따로 빼서 mush_test.csv 로 저장한다. 
mush_test <- mushroom[8123, ]
mush_test 
write.csv( mush_test, "mush_test.csv",row.names=FALSE )

# 3. 8124 독버섯 데이터를 훈련 데이터에서 제외 시키시오 !
nrow(mushroom)
mushrooms <- mushroom[ -8123,  ] 
nrow(mushrooms)

# 4. mushrooms 데이터를 훈련 데이터와 테스트 데이터로 나눈다 ( 훈련 데이터는 75%,  테스트 데이터는 25% )
set.seed(1)
dim(mushrooms) # 차원 확인

train_cnt <- round( 0.75*dim(mushrooms)[1] )
train_cnt 

train_index <- sample( 1:dim(mushrooms)[1], train_cnt, replace=F)

mushrooms_train <- mushrooms[ train_index,  ]
mushrooms_test <- mushrooms[- train_index,  ] 

nrow(mushrooms_train)  #  6092
nrow(mushrooms_test)    #  2031 

str(mushrooms_train)

# 5. 나이브 베이즈 알고리즘으로 독버섯과 일반 버섯을 분류하는 모델을 생성한다.
library(e1071) #        모든 컬럼들
#                          ↓
model1 <- naiveBayes(type~ . ,  data=mushrooms_train)
#                     ↑
#                   라벨 컬럼명 
model1

```
    4단계 : 모델 성능평가 -> 이원교차표를 통해 정확도 확인. 
                ex) 병원모델예측 -> 양성/ 실제 -> 악성 ≫ 이런 모델 사용 불가
```

In [ ]:
# 6. 위에서 만든 모델과 테스트 데이터를 가지고 독버섯과 일반버섯을 잘 분류하는지 예측해 본다.
result1 <- predict( model1, mushrooms_test[  , -1] ) # label 제외
result1 

# 7. 이원 교차표를 그려서 최종 분류 결과를 확인한다. 
library(gmodels)
CrossTable( mushrooms_test[  ,1], result1) 
#                   ↑                ↑
#                  실제            예측 

```
    5단계 : 모델 성능개선 -> 나이브 베이즈 모델의 성능을 높이는 방법은 라플라스 값을 알아내는 것                            
```

In [ ]:
# 8. 위의 모델의 성능을 올리시오 !
model2 <- naiveBayes(type~ . ,  data=mushrooms_train, laplace=0.0004)

result2 <- predict( model2, mushrooms_test[ , -1] )

CrossTable( mushrooms_test[ ,1], result2) 

### ※ 위의 모델에  별도로 구분해 놓은 테스트 데이터 한개(독버섯) 8123 번 데이터를 넣어서 독버섯인지 정상인지 확인하시오 ! 

In [ ]:
result3 <- predict( model2, mush_test )
result3

### ※ 문제228. 어제 사용한 k값을 알아내는 코드를 이용해서 독버섯 분류 나이브 베이즈 모델에 가장 적합한 Laplace 값이 얼마인지 알아내시오
    0.001부터 시작

In [3]:
temp<-c()
for ( i in 1:100 ) {
  model_test_pred <- naiveBayes(type~ . ,  data=mushrooms_train, laplace=i/100000)
  rs <- predict( model_test_pred, mushrooms_test[,-1])
  g2 <- CrossTable(x=mushrooms_test[,1], y=rs, chisq=FALSE)
  g3 <- g2$prop.tbl[1] + g2$prop.tbl[4]
  temp<-append(temp, g3 )
}
temp
plot(temp, type='l', col='red')

ERROR: Error in naiveBayes(type ~ ., data = mushrooms_train, laplace = i/1e+05): 함수 "naiveBayes"를 찾을 수 없습니다
